# Automated evaluation of FinanceBench using ROUGE-SU

Currently, there are not that many automated eval options with LLMs.


**Insert more text here
- Goals
- Approach
- Outline

## Preprocessing
We can use NLTK to remove stop words and remap the questions into a `List[str]` type

In [6]:
import pandas as pd
from openai import OpenAI
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from functools import partial
import rouge_metric
from tqdm import tqdm
import time


#Data processing
df = pd.read_csv("financebench_sample_150.csv")[["question", "answer"]]
stop_words = list(set(stopwords.words('english')))
def preprocess(sequence: str):
    returnVal = [w for w in word_tokenize(sequence) if not w.lower() in stop_words]
    #returnVal = [''.join(c for c in s if c not in string.punctuation) for s in returnVal]
    return list(filter(None, returnVal))

df["answer"] = df["answer"].apply(preprocess)
df.head()

,question,answer
0,What is the FY2018 capital expenditure amount ...,"[$, 1577.00]"
1,Assume that you are a public equities analyst....,"[$, 8.70]"
2,Is 3M a capital-intensive business based on FY...,"[,, company, managing, CAPEX, Fixed, Assets, p..."
3,What drove operating margin change as of FY202...,"[Operating, Margin, 3M, FY2022, decreased, 1.7..."
4,"If we exclude the impact of M&A, which segment...","[consumer, segment, shrunk, 0.9, %, organicall..."


In [24]:
print(custom_completion(df['question'][1]))

I currently do not have access to specific financial data for 3M for the year end FY2018, including the net PP&E (Property, Plant, and Equipment). To obtain this information, please refer to 3M's official financial statements for FY2018, which can typically be found in their annual report or on their investor relations website.


In [16]:
#OpenAI client, default GPT4
default_client = OpenAI()
custom_client = OpenAI(api_key="NONE", base_url="https://finsearch-l35slsdlnq-uc.a.run.app")
def gpt_wrap_message(client: OpenAI, query: str):
    return client.chat.completions.create(
        model = "gpt-4-turbo",
        messages = [
            {"role": "user", "content": query}
        ]
        ).choices[0].message.content

def customgpt_wrap_message(client: OpenAI, query: str):
    return client.chat.completions.create(
        model = "gpt-4-turbo",
        messages = [
            {"role": "user", "content": query}
        ]
        ).choices[0].content


#Check the progress meter for each of our df.apply() methods
tqdm.pandas()

#Fix argument 1 with our OpenAI clients
default_completion = partial(gpt_wrap_message, default_client)
custom_completion = partial(customgpt_wrap_message, custom_client)

#df['gpt_answers'] = df['question'].progress_apply(default_completion)
#df['custom_answers'] = df['question'].progress_apply(custom_completion)

In [27]:
df.to_pickle("./financebench_answers.pkl")

### Start Here if data is loaded from pickle

In [3]:
new_df = pd.read_pickle('./financebench_answers.pkl')

In [18]:
custom_results = []
for i in tqdm(range(len(new_df))):
    custom_results.append(custom_completion(df['question'][i]))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [20:14<00:00,  8.10s/it]


In [24]:
new_df2 = new_df.assign(custom_answers=custom_results)
new_df2.to_pickle("./financebench_final.pkl")

In [25]:
third_df = pd.read_pickle('./financebench_final.pkl')

In [30]:
third_df['custom_answers'][0]

'The FY2018 capital expenditure amount for 3M in USD millions was 1,550 million. This figure is derived from the snippet stating that 3M reported a capital expenditure of 1.55 billion in the previous year. Since 1 billion equals 1,000 million, the conversion from 1.55 billion to million is 1,550 million.'

### Running Metrics

In [ ]:
from rouge_metric import PerlRouge

rouge = PerlRouge(rouge_n_max=3, rouge_l=True, rouge_w=True,
    rouge_w_weight=1.2, rouge_s=True, rouge_su=True, skip_gap=4)